<a href="https://colab.research.google.com/github/stvngo/Algoverse-AI-Model-Probing/blob/main/Steven_Model_%26_Dataset_Verification_with_Activation_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Promting and Model Probing #
Steven's prompts for illiciting CoT reasoning responses from models like **Gwen-3**, **DeepSeek-R1**, **Llama-2**, etc.

**Model Paths**
*   [Microsoft Phi-4](https://huggingface.co/microsoft/phi-4): microsoft/phi-4
*   [Meta Llama-2](https://huggingface.co/meta-llama/Llama-2-7b): meta-llama/Llama-2-7b
*   [Qwen 3 0.6B](https://huggingface.co/Qwen/Qwen3-0.6B): Qwen/Qwen3-0.6B
*   [DeepSeek R1 Distill Qwen 1.5B](https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B): deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B




In [ ]:
# install necessary libraries
!pip install --upgrade datasets
!pip install --upgrade transformers
!pip install --upgrade einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

# Testing and Prompting on Qwen 3 0.6B with Qwen3 PTS dataset

In [ ]:
# Download using git (if it's a git repository)
# !git clone https://huggingface.co/datasets/codelion/Qwen3-0.6B-pts

**Load Dataset**

My load_dataset() function was not working at the time, had to do troubleshifting and got it to work on a pandas dataframe.

In [ ]:
# First, let's debug the datasets library
print("=== Debugging datasets library ===")
try:
    from datasets import load_dataset
    print("✓ datasets library imported successfully")

    # Test with a simple, known dataset
    print("Testing with a simple dataset (squad)...")
    test_dataset = load_dataset("squad", split="train[:5]")
    print(f"✓ Test dataset loaded: {len(test_dataset)} examples")

except Exception as e:
    print(f"✗ Error with datasets library: {e}")
    print("The datasets library seems to have issues. Let's try alternatives...")

# Alternative 1: Try downloading files directly
print("\n=== Alternative 1: Direct file download ===")
try:
    from huggingface_hub import hf_hub_download
    import pandas as pd

    # List available files first
    from huggingface_hub import list_repo_files
    files = list_repo_files("codelion/Qwen3-0.6B-pts", repo_type="dataset")
    print(f"Available files: {files}")

    # Try downloading a data file
    data_files = [f for f in files if f.endswith(('.parquet', '.json', '.jsonl', '.csv'))]
    if data_files:
        print(f"Found data files: {data_files}")
        file_path = hf_hub_download(
            repo_id="codelion/Qwen3-0.6B-pts",
            filename=data_files[0],
            repo_type="dataset"
        )

        # Load based on file type
        if data_files[0].endswith('.parquet'):
            df = pd.read_parquet(file_path)
        elif data_files[0].endswith('.json'):
            df = pd.read_json(file_path)
        elif data_files[0].endswith('.jsonl'):
            df = pd.read_json(file_path, lines=True)
        elif data_files[0].endswith('.csv'):
            df = pd.read_csv(file_path)

        print(f"✓ Data loaded as pandas DataFrame: {len(df)} rows")
        print(f"Columns: {list(df.columns)}")

        # Show first example
        if len(df) > 0:
            print(f"\nFirst example:")
            for col in df.columns:
                value = df.iloc[0][col]
                if isinstance(value, str) and len(value) > 100:
                    print(f"  {col}: {value[:100]}...")
                else:
                    print(f"  {col}: {value}")

        # Store as our dataset
        dataset = df

except Exception as e:
    print(f"✗ Direct download failed: {e}")

=== Debugging datasets library ===
✓ datasets library imported successfully
Testing with a simple dataset (squad)...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✗ Error with datasets library: Invalid pattern: '**' can only be an entire path component
The datasets library seems to have issues. Let's try alternatives...

=== Alternative 1: Direct file download ===
Available files: ['.gitattributes', 'README.md', 'gsm8k_pivotal_tokens.jsonl', 'pivotal_tokens.jsonl']
Found data files: ['gsm8k_pivotal_tokens.jsonl', 'pivotal_tokens.jsonl']


gsm8k_pivotal_tokens.jsonl: 0.00B [00:00, ?B/s]

✓ Data loaded as pandas DataFrame: 405 rows
Columns: ['model_id', 'query', 'pivot_context', 'pivot_token', 'pivot_token_id', 'prob_before', 'prob_after', 'prob_delta', 'is_positive', 'task_type', 'dataset_id', 'dataset_item_id', 'timestamp']

First example:
  model_id: Qwen/Qwen3-0.6B
  query: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much ...
  pivot_context: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much ...
  pivot_token: A
  pivot_token_id: 32
  prob_before: 0.68
  prob_after: 0.0
  prob_delta: -0.68
  is_positive: False
  task_type: generic
  dataset_id: openai/gsm8k
  dataset_item_id: 1
  timestamp: 2025-05-13 14:00:26


**Dataset**

Use pre-identified pivotal tokens from the dataset as groudn truth labels for training linear probes.

In [ ]:
dataset

,model_id,query,pivot_context,pivot_token,pivot_token_id,prob_before,prob_after,prob_delta,is_positive,task_type,dataset_id,dataset_item_id,timestamp
0,Qwen/Qwen3-0.6B,Weng earns $12 an hour for babysitting. Yester...,Weng earns $12 an hour for babysitting. Yester...,A,32,0.68,0.00,-0.68,False,generic,openai/gsm8k,1,2025-05-13 14:00:26
1,Qwen/Qwen3-0.6B,Weng earns $12 an hour for babysitting. Yester...,Weng earns $12 an hour for babysitting. Yester...,,220,0.80,0.40,-0.40,False,generic,openai/gsm8k,1,2025-05-13 14:34:13
2,Qwen/Qwen3-0.6B,Weng earns $12 an hour for babysitting. Yester...,Weng earns $12 an hour for babysitting. Yester...,A,32,0.68,0.00,-0.68,False,generic,openai/gsm8k,1,2025-05-13 14:51:23
3,Qwen/Qwen3-0.6B,Weng earns $12 an hour for babysitting. Yester...,Weng earns $12 an hour for babysitting. Yester...,A,32,0.68,0.00,-0.68,False,generic,openai/gsm8k,1,2025-05-13 15:31:21
4,Qwen/Qwen3-0.6B,Mark has a garden with flowers. He planted pla...,Mark has a garden with flowers. He planted pla...,To,2014,0.10,0.76,0.66,True,generic,openai/gsm8k,5,2025-05-13 16:00:23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,Qwen/Qwen3-0.6B,James has a rainwater collection barrel. For ...,James has a rainwater collection barrel. For ...,We,1654,0.56,0.98,0.42,True,generic,openai/gsm8k,96,2025-05-19 09:23:58
401,Qwen/Qwen3-0.6B,James has a rainwater collection barrel. For ...,James has a rainwater collection barrel. For ...,Round,27497,0.20,0.44,0.24,True,generic,openai/gsm8k,96,2025-05-19 09:33:14
402,Qwen/Qwen3-0.6B,James has a rainwater collection barrel. For ...,James has a rainwater collection barrel. For ...,Answer,16141,0.58,0.84,0.26,True,generic,openai/gsm8k,96,2025-05-19 09:33:14
403,Qwen/Qwen3-0.6B,James has a rainwater collection barrel. For ...,James has a rainwater collection barrel. For ...,Round,27497,0.20,0.44,0.24,True,generic,openai/gsm8k,96,2025-05-19 09:47:05


In [ ]:
# JSON string formatting
dataset_json = dataset.to_dict('records')
dataset_json[0:5]

[{'model_id': 'Qwen/Qwen3-0.6B',
  'query': 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?',
  'pivot_context': 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn? \n\n',
  'pivot_token': 'A',
  'pivot_token_id': 32,
  'prob_before': 0.68,
  'prob_after': 0.0,
  'prob_delta': -0.68,
  'is_positive': False,
  'task_type': 'generic',
  'dataset_id': 'openai/gsm8k',
  'dataset_item_id': 1,
  'timestamp': Timestamp('2025-05-13 14:00:26')},
 {'model_id': 'Qwen/Qwen3-0.6B',
  'query': 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?',
  'pivot_context': 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn? To solve the problem, you should multiply',
  'pivot_token': ' ',
  'pivot_token_id': 220,
  'prob_before': 0.8,
  'prob_after': 0.4,


In [ ]:
# # create test, train, and validation split
# import torch.utils.data.random_split as random_split
# torch.manual_seed(42)
# train_size = int(0.8 * len(dataset))
# test_size = len(dataset) - train_size
# train_dataset, test_dataset = random_split(dataset_json, [train_size, test_size])

# query_array = dataset.get("query").unique()

**Model Loading**

Load the model with configurations for interp work, including disabled gradients and activation extraction.

Notes:


*   Padding adds special tokens to sequences to make them all the same length in a batch. Important when processing multiple sequences in batches for efficiency, extracting activations from sequences of different lengths, and aligning token positions across examples

In [ ]:
# import necessary packages
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# disable gradients for efficiency (only forward passes)
torch.set_grad_enabled(False)

# set default device to CUDA (gpu)
# torch.set_default_device("cuda")

# check device availability (save resources)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# model name
model_name = "Qwen/Qwen3-0.6B"

# load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype="auto",
                                             trust_remote_code=True,
                                             output_hidden_states=True) # access internal activations

# move model to the selected device
model.to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)


# set model to evaluation mode
model.eval()

# add padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"✓ Model loaded successfully")
print(f"Model device: {next(model.parameters()).device}")
print(f"Model dtype: {next(model.parameters()).dtype}")
print(f"Number of layers: {len(model.model.layers)}")

Using device: cuda


config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

✓ Model loaded successfully
Model device: cuda:0
Model dtype: torch.bfloat16
Number of layers: 28


**Tokenization Validation**

Critical function that checks whether the pivot tokens in the dataset align with how the model actually tokenizes the text. This is essential since misalignment would invalidate interpretability results. (If there's a mismatch, the pivotal token labels are wrong)

Notes:


*   tokenizer.encode(): simpler, lower-level method that returns token IDs as a list of integers.


*   tokenizer(): full-featured, high-level method that returns a dictionary with multiple components, i.e. token ids, attention mask, token type ids, etc. Use for actual model forward passes to get the attention masks. Needed later because when extracting activations at specific token positions, need to know which positions are actual tokens vs padding.


*   tokenizer.decode(): reverse of tokenization, converts token IDs back into human-readable text





In [ ]:
# token validation

def validate_tokenization(example_idx: int = 0):
  """Validate that pivot tokens align with model tokenization"""
  row = dataset.iloc[example_idx]

  # get the pivot context and token
  pivot_context = row["pivot_context"]
  pivot_token = row["pivot_token"]

  # tokenize the context
  context_tokens = tokenizer.encode(pivot_context, return_tensors="pt")

  # tokenize the context + pivot
  full_sequence = pivot_context + pivot_token
  full_tokens = tokenizer.encode(full_sequence, return_tensors="pt")

  # check if adding the pivot token matches expectations
  context_length = context_tokens.shape[1]
  full_length = full_tokens.shape[1]

  print(f"Example {example_idx}:")
  print(f"  Context length: {context_length} tokens")
  print(f"  Full sequence length: {full_length} tokens")
  print(f"  Pivot token: '{pivot_token}'")
  print(f"  Context ends with: '{tokenizer.decode(context_tokens[0][-3:])}'") # 2-d tensor, use [0] first
  print(f"  Full sequence ends with: '{tokenizer.decode(full_tokens[0][-3:])}'")

  # Extract what the model thinks is the next token after context
  if full_length > context_length:
      predicted_next_tokens = full_tokens[0][context_length:]
      predicted_next_text = tokenizer.decode(predicted_next_tokens)
      print(f"  Model's next token(s): '{predicted_next_text}'")
      print(f"  Matches pivot token: {predicted_next_text.strip() == pivot_token.strip()}")

  return context_tokens, full_tokens

context_tokens, full_tokens = validate_tokenization(example_idx=0)
print(context_tokens)
print(full_tokens) # 32 is the last token encoded
print(f'Dataset pivot token id: {dataset.get("pivot_token_id").iloc[0]}') # should match 32 from the dataset

Example 0:
  Context length: 32 tokens
  Full sequence length: 33 tokens
  Pivot token: 'A'
  Context ends with: ' earn? 

'
  Full sequence ends with: '? 

A'
  Model's next token(s): 'A'
  Matches pivot token: True
tensor([[   54,   826, 63759,   400,    16,    17,   458,  6460,   369, 70583,
         14810,    13, 60033,    11,  1340,  1101,  1521,   220,    20,    15,
          4420,   315, 70583, 14810,    13,  2585,  1753,  1521,  1340,  7232,
            30,  4710]])
tensor([[   54,   826, 63759,   400,    16,    17,   458,  6460,   369, 70583,
         14810,    13, 60033,    11,  1340,  1101,  1521,   220,    20,    15,
          4420,   315, 70583, 14810,    13,  2585,  1753,  1521,  1340,  7232,
            30,  4710,    32]])
Dataset pivot token id: 32


**Generation Settings**

Settings for reproducing the model's behavior (validation), extracting internal representations at the right positions during forward passes, and verifying model setup matches dataset creation setup.

Notes:
*   max_new_tokens=1: only
*   do_sample=False: always pick highest probable token, ensures reproducibility
*   pad_token_id + eos_token_id: what tokens represent padding and end-of-sequence for sequence boundaries
*   output_attentions=False: return attention weights from all layers (not needed, save resources)
*   **output_hidden_states=True: returns activations from ALL LAYERS, needed to train linear probes**
*   return_dict_in_generate=True: aceess to hidden states during generation



In [ ]:
# generation settings
generation_config = {
    'max_new_tokens': 1,  # we're only interested in the next token for probability calculation
    'do_sample': False,   # deterministic generation
    'temperature': 1.0,
    'top_p': 1.0,
    'pad_token_id': tokenizer.pad_token_id,
    'eos_token_id': tokenizer.eos_token_id,
    'output_hidden_states': True,  # CRITICAL for activation extraction
    'output_attentions': False,    # not needed for now, saves memory
    'return_dict_in_generate': True,
    'use_cache': False,  # disable caching for cleaner memory usage
}

**Helper functions**

Use **calculate_next_token_probability()** to verify the model matches the dataset.
Use **get_model_activations** to extract features at pivot positions.

Notes:

*   Activations should tell you what the model was thinking at the moment before it generated each token. The linear probes will learn to recognize patterns in these "thoughts" that predict whether the next token will be pivotal for the final answer.

*   Use pivot/non-pivot (is_pivotal) labels to train linear probe, and activations as input features (logistic regression)

*   with torch.no_grad(): disables gradients for everything inside the block (we never need gradients)
*   .logits: raw, unnormalized predictions from the model with shape (batch_size, sequence_length, vocab_size). For each position, get a score for every possible token in the vocabulary
*   .hidden_states: internal representations from each layer of the transformer. len(hidden_states) = 29, n layers + 1 unembedding layer

*   model.generate() is different from **model()**: model.generate() repeatedly calls the model to generate new tokens one by one, **model()** only runs the model **ONCE** on the input **(should be the context before the pivot)**, and processes all tokens in the sequence at once, extracting activations from every position. **Perfect because we need to extract the activations at the exact position right before each pivot token.**

In [ ]:
# helper functions
def get_model_activations(input_ids: torch.Tensor, attention_mask: torch.Tensor):
    """Extract activations from all layers of the model"""
    # Ensure model is on the same device as input tensors
    model.to(input_ids.device)

    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            use_cache=False
        )

    # Extract hidden states from all layers
    hidden_states = outputs.hidden_states  # Tuple of (batch_size, seq_len, hidden_size)

    activations = {}
    for layer_idx, layer_activations in enumerate(hidden_states):
        activations[f'layer_{layer_idx}'] = layer_activations

    return activations # returns: {'layer_0': tensor, 'layer_1': tensor, ...}

# for debugging and exploration
def calculate_next_token_probability(context: str, target_token: str) -> float:
    """Calculate probability of target token given context"""
    # Tokenize context
    context_tokens = tokenizer.encode(context, return_tensors="pt").to(device)

    # Get model logits
    with torch.no_grad():
        outputs = model(context_tokens)
        logits = outputs.logits[0, -1, :]  # Last token logits
        print(f"Vector of logits for next token: {logits}")

    # Get probability distribution
    probs = torch.softmax(logits, dim=-1)
    print(f"Vector of probabilities for next token: {probs}")

    # Tokenize target token to get its ID
    target_token_id = tokenizer.encode(target_token, add_special_tokens=False)[0]
    print(f"Target token id: {target_token_id}")

    # Return probability
    return probs[target_token_id].item()

print(f"Probability of 'jumps': {calculate_next_token_probability(context='The quick brown fox', target_token='jumps')}")
# very low probability

Vector of logits for next token: tensor([ 9.2500, 10.9375,  8.8125,  ..., -3.3594, -3.3594, -3.3594],
       device='cuda:0', dtype=torch.bfloat16)
Vector of probabilities for next token: tensor([3.6210e-06, 1.9550e-05, 2.3395e-06,  ..., 1.2108e-11, 1.2108e-11,
        1.2108e-11], device='cuda:0', dtype=torch.bfloat16)
Target token id: 73
Probability of 'jumps': 3.0100345611572266e-06


Test cases for the above functions

In [ ]:
def check_activation_extraction(num_examples_to_check: int = 3):
    """Checks if the get_model_activations function extracts activations correctly."""
    print(f"Checking activation extraction for {num_examples_to_check} examples:")

    for i in range(min(num_examples_to_check, len(dataset))):
        row = dataset.iloc[i]
        pivot_context = row["pivot_context"]

        # Tokenize the context
        inputs = tokenizer(pivot_context, return_tensors="pt", padding=True, truncation=True)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        # Get activations
        activations = get_model_activations(input_ids, attention_mask)

        print(f"\nExample {i}:")
        print(f"  Context: '{pivot_context[:100]}...'")
        print(f"  Number of layers with activations: {len(activations)}")

        # Check shape of activations for a few layers
        for layer_idx in [0, 5, 10, len(model.model.layers)-1]: # Check first, middle and last layers
            if f'layer_{layer_idx}' in activations:
                act_shape = activations[f'layer_{layer_idx}'].shape
                print(f"    Layer {layer_idx} activation shape: {act_shape}")
            else:
                print(f"    Layer {layer_idx} activations not found.")

check_activation_extraction()

Checking activation extraction for 3 examples:

Example 0:
  Context: 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much ...'
  Number of layers with activations: 29
    Layer 0 activation shape: torch.Size([1, 32, 1024])
    Layer 5 activation shape: torch.Size([1, 32, 1024])
    Layer 10 activation shape: torch.Size([1, 32, 1024])
    Layer 27 activation shape: torch.Size([1, 32, 1024])

Example 1:
  Context: 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much ...'
  Number of layers with activations: 29
    Layer 0 activation shape: torch.Size([1, 39, 1024])
    Layer 5 activation shape: torch.Size([1, 39, 1024])
    Layer 10 activation shape: torch.Size([1, 39, 1024])
    Layer 27 activation shape: torch.Size([1, 39, 1024])

Example 2:
  Context: 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much ...'
  Number of layers with activation

In [ ]:
# small test case for get_model_activations
print("=== Testing get_model_activations with one example ===")

# get the first example from the dataset
example_row = dataset.iloc[0]
pivot_context = example_row["pivot_context"]

print(f"Using context: '{pivot_context[:100]}...'")

# Tokenize the context
inputs = tokenizer(pivot_context, return_tensors="pt", padding=True, truncation=True)
input_ids = inputs["input_ids"].to(device)
attention_mask = inputs["attention_mask"].to(device)

print(f"Input IDs shape: {input_ids.shape}")
print(f"Attention Mask shape: {attention_mask.shape}")

# Get activations
activations = get_model_activations(input_ids, attention_mask)

print(f"Successfully extracted activations from {len(activations)} layers.")

# Print shape of activations for a few layers to verify
for layer_idx in [0, 10, len(model.model.layers)-1]:
    if f'layer_{layer_idx}' in activations:
        act_shape = activations[f'layer_{layer_idx}'].shape
        print(f"  Layer {layer_idx} activation shape: {act_shape}")
    else:
        print(f"  Layer {layer_idx} activations not found.")

=== Testing get_model_activations with one example ===
Using context: 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much ...'
Input IDs shape: torch.Size([1, 32])
Attention Mask shape: torch.Size([1, 32])
Successfully extracted activations from 29 layers.
  Layer 0 activation shape: torch.Size([1, 32, 1024])
  Layer 10 activation shape: torch.Size([1, 32, 1024])
  Layer 27 activation shape: torch.Size([1, 32, 1024])


# Next Steps

The next step is to **extract the activations for the entire dataset** and prepare the data for training the linear probes. This involves iterating through each example, getting the activations at the relevant token position, and **collecting the corresponding labels**.

Plan for preparing the data:

1.   Iterate through the dataset: Loop through each row of the dataset.
2.   Tokenize context and identify pivot token position: For each example, tokenize the pivot_context and determine the index of the pivot token within the tokenized sequence.
3.   Extract activations: Use the get_model_activations function to get the activations for the tokenized pivot_context.
4.   Select pivot position activations: From the extracted activations, select the activations corresponding to the position right before the pivot token.
5.   Collect labels: Get the **is_positive** label for each example.
6.   Store extracted data: Store the extracted activations and the corresponding labels in a suitable format (e.g., NumPy arrays or PyTorch tensors).
7.   Finish task: Summarize the data preparation process and confirm the data is ready for training.

See: https://carpentries-incubator.github.io/fair-explainable-ml/5c-probes.html